In [19]:
# Tools for inversion
import math
import random

def EX_GCD(a, b, arr):  # 扩展欧几里得
    if b == 0:
        arr[0] = 1
        arr[1] = 0
        return a
    g = EX_GCD(b, a % b, arr)
    t = arr[0]
    arr[0] = arr[1]
    arr[1] = t - int(a / b) * arr[1]
    return g


def ModReverse(a, n):  # ax=1(mod n) 求a模n的乘法逆x
    arr = [0, 1, ]
    gcd = EX_GCD(a, n, arr)
    if gcd == 1:
        return (arr[0] % n + n) % n
    else:
        return -1


In [20]:
# Montgomery and Plantard modular arithmetic
import numpy as np
QINV_MONT = 57857
Q=7681
MONT=4088
# For Plantard Multiplication
QINV_PLANT = 2340676097
PLANT=2112
R=2**32
r=2**16
alpha = 2
alpha2 = 2**alpha

MONT2inv=ModReverse((MONT*MONT)%Q,Q)
PLANT2inv=ModReverse((PLANT*PLANT)%Q,Q)
print(MONT2inv,PLANT2inv)

def mont_mul(a,b):
	c = np.int32(a*b)
	u = np.int16(c*QINV_MONT)
	t = np.int32(u*Q)
	t = (c-t)>>16
	return t

def plant_red(a):
	t = (((((a*QINV_PLANT) % R)//r)+alpha2)*Q//r)
	return t

def plant_mul(a, b):
	return plant_red(a*b)

# zeta=zeta*QINV_PLANT%R
def plant_mul_ntt(a, zeta):
	t = (((((a*zeta) % R)//r)+alpha2)*Q//r)
	return t

# Correctness
result=True
for i in range(0,10000):
	a=random.randint(0,Q)
	b=random.randint(0,Q)
	cm=mont_mul(a,b)
	cp=plant_mul(a,b)
	cpm=mont_mul(cp,PLANT)
	if((cpm%Q)!=(cm%Q)):
		print(i, cpm, cp)
		result = False
if(result):
	print("Mont and Plant arithmetic Correct!")
else:
	print("Mont and Plant arithmetic Fail!")


3495 2235
Mont and Plant arithmetic Correct!


In [21]:
# Conversion between Montgomery and Plantard domain
P2M=-r
M2P=ModReverse(-r,Q)
def plant2mont(a):
	return (a*P2M) % Q
def mont2plant(a):
	return (a*M2P) % Q

def plant2mont1(a):
	return mont_mul(a, MONT) % Q
def mont2plant1(a):
	return plant_mul(a, PLANT)


In [22]:
# Conversion between Montgomery and Plantard Domain;

import random

a=random.randint(0,Q)
b=random.randint(0,Q)
print(a,b)
c=a*b
print("c=a*b=",c)
c_mont=mont_mul(a,b)
print("c=Mont(a,b)=",c_mont%Q)
c_plant=plant_mul(a, b)
print("c=Plant(a,b)=",c_plant)
print("plant2mont", 
plant2mont(c_plant))
print("mont2plant", mont2plant(c_mont))


2501 5297
c=a*b= 13247797
c=Mont(a,b)= 706
c=Plant(a,b)= 2123
plant2mont 706
mont2plant 2123


In [23]:
# zetas in Montgomery
zetas_mont = [-2731, -308, -1252, -2652, -1463, 2765, 2162, -1734, 221, -2586, -762, -2970, 2901, 2235, 1015, -338, 848, -3771, -630, 3653, -1398, -2062, 1727, 2800, 2327, 784, 3724, -1452, 2631, 1606, -3893, -2896, -3236, 1963, -277, 9, 190, 2434, 40, 2938, -3801, 789, -2013, -3068, -1614, -1029, 873, -3855, -3324, 108, 513, 427, 480, -3149, -1516, -2280, -1113, 1589, 1787, -474, 2795, -174, 3014, -3675, -853, -2357, 2246, -1709, 4033, -2183, -768, 1966, 2314, -1323, 3317, 530, 1801, -1750, 3209, -2794, -2707, 2029, 3877, 3257, -2077, -1624, -33, -3576, -3201, -1227, 3773, 1763, -2893, 1425, 1008, 300, 1236, -2924, 1473, 1810, 1513, -2933, -490, -1426, -1444, -64, -304, -822, 569, 3004, -1093, 3058, -787, -2886, -2187, 1818, -25, 84, 399, 2039, 298, -1923, 467, 2425, -3426, -471, -317, -2929, -2731, -1270, -2192, 3371, 2890, 1043, 3034, -2206, 2420, 1086, 1318, -3814, -294, 3753, -3296, -2444, 2553, -3355, 1346, 1315, -1256, 1455, -2690, -1715, 224, 2627, 2877, -1064, -2833, -1849, -3022, 15, -3030, -3645, -3872, 2871, -838, 665, -2602, 140, 1079, -1782, 3057, -3205, -239, 2032, 1971, -785, 1370, -3067, 2714, -2667, 24, 3299, -1612, -114, -2744, -3377, -2599, -2328, 2060, -2313, 2455, -2104, -3272, 855, 2141, 180, 3800, 2594, 800, -2688, 790, 418, -1855, 88, -1556, 2463, 2098, -290, -3261, 2047, 122, 2048, -1997, -3429, -2846, -3956, 454, 318, -2330, 1684, -1147, 1396, -1050, 3528, 1677, 2968, -1264, -2205, -464, 2788, -2119, 2204, 1844, -1280, 1601, -1078, 3699, -1368, 1183, -288, -378, -3953, 2346, -2045, -500, 1680, 299, 2375, -1721, -55, 1659, 2414, 609, -1739, 1341, 2868]
zetas_inv_mont = [-2868, -1341, 1739, -609, -2414, -1659, 55, 1721, -2375, -299, -1680, 500, 2045, -2346, 3953, 378, 288, -1183, 1368, -3699, 1078, -1601, 1280, -1844, -2204, 2119, -2788, 464, 2205, 1264, -2968, -1677, -3528, 1050, -1396, 1147, -1684, 2330, -318, -454, 3956, 2846, 3429, 1997, -2048, -122, -2047, 3261, 290, -2098, -2463, 1556, -88, 1855, -418, -790, 2688, -800, -2594, -3800, -180, -2141, -855, 3272, 2104, -2455, 2313, -2060, 2328, 2599, 3377, 2744, 114, 1612, -3299, -24, 2667, -2714, 3067, -1370, 785, -1971, -2032, 239, 3205, -3057, 1782, -1079, -140, 2602, -665, 838, -2871, 3872, 3645, 3030, -15, 3022, 1849, 2833, 1064, -2877, -2627, -224, 1715, 2690, -1455, 1256, -1315, -1346, 3355, -2553, 2444, 3296, -3753, 294, 3814, -1318, -1086, -2420, 2206, -3034, -1043, -2890, -3371, 2192, 1270, 2731, 2929, 317, 471, 3426, -2425, -467, 1923, -298, -2039, -399, -84, 25, -1818, 2187, 2886, 787, -3058, 1093, -3004, -569, 822, 304, 64, 1444, 1426, 490, 2933, -1513, -1810, -1473, 2924, -1236, -300, -1008, -1425, 2893, -1763, -3773, 1227, 3201, 3576, 33, 1624, 2077, -3257, -3877, -2029, 2707, 2794, -3209, 1750, -1801, -530, -3317, 1323, -2314, -1966, 768, 2183, -4033, 1709, -2246, 2357, 853, 3675, -3014, 174, -2795, 474, -1787, -1589, 1113, 2280, 1516, 3149, -480, -427, -513, -108, 3324, 3855, -873, 1029, 1614, 3068, 2013, -789, 3801, -2938, -40, -2434, -190, -9, 277, -1963, 3236, 2896, 3893, -1606, -2631, 1452, -3724, -784, -2327, -2800, -1727, 2062, 1398, -3653, 630, 3771, -848, 338, -1015, -2235, -2901, 2970, 762, 2586, -221, 1734, -2162, -2765, 1463, 2652, 1252, 1568]


In [24]:
# NTT based on Montgomery
# Mont NTT Correct. 
def ntt_mont(a):
	for j in range(0, 384):
		t=mont_mul(zetas_mont[1],a[j+384])
		a[j+384]=np.int16(a[j]+a[j+384]-t)%Q
		a[j] = np.int16(a[j]+t)%Q
	# print("After 1 layer:",a)
	k=2
	len=192
	while len>=3:
		# print(len)
		start=0
		while start<768:
			zeta=zetas_mont[k]
			k=k+1
			j=start
			while j<start+len:
				t = mont_mul(zeta, a[j+len])
				a[j+len] = np.int16(a[j]-t)%Q
				a[j] = np.int16(a[j]+t) % Q
				j=j+1
			start=j+len
		len=len>>1
	return
# Mont INVNTT Correct.
def invntt_mont(a):
	k = 0
	len = 3
	while len <=192:
		# print(len)
		start = 0
		while start < 768:
			zeta = zetas_inv_mont[k]
			k = k+1
			j = start
			while j < start+len:
				t=a[j]
				a[j]=np.int16(t+a[j+len])%Q
				a[j+len] = np.int16(t-a[j+len])
				a[j+len] = mont_mul(zeta, a[j+len])
				j = j+1
			start = j+len
		len = len << 1
	
	# i=0
	# while i<768:
	# 	a[i]=mont_mul(a[i],(MONT*MONT)%Q)%Q
	# 	i += 1
	# print("before final layer:", a)
	for j in range(0, 384):
		t = np.int16(a[j]-a[j+384])
		a[j] = np.int16(a[j]+a[j+384])
		t = mont_mul(zetas_inv_mont[254],t)
		# print(j, ": ", t%Q)
		a[j] = np.int16(a[j]-t)
		a[j] = mont_mul((1<<24)%Q, a[j])
		a[j+384]=mont_mul((1<<25)%Q, t)
	# print("After 1 layer:",a)
	return

In [25]:
# zetas in Plantard
def init_ntt_c_plant_from_mont(zetas, type):
	plant_ntt_asm = []
	plant_ntt_asm_prime = []
	Mont_inv = ModReverse(MONT, Q)
	for zeta in zetas:
		t = PLANT*zeta*Mont_inv % Q
		plant_ntt_asm.append(t)
	if type == 1:
		# for inverse NTT. Finalize the final two twiddle factors in the final layer as (Plant^2*256^-1)%M and (Plant^2*128^-1)%M
		final = PLANT*(PLANT*(Q-1)*((Q-1)/256) % Q) % Q
		plant_ntt_asm.append(int(final))
		final = PLANT*(PLANT*(Q-1)*((Q-1)/128) % Q) % Q
		plant_ntt_asm.append(int(final))
		# plant_ntt_asm[-1] = (zetas[-1]*PLANT*Mont_inv) % Q
		# print(plant_ntt_asm[-1])
	for zeta in plant_ntt_asm:
		# zeta*Mprime_plant %R
		t = (zeta*QINV_PLANT) % R
		plant_ntt_asm_prime.append(t)
	return plant_ntt_asm_prime

# init_ntt_c_plant_from_mont(zetas_mont,0)
# init_ntt_c_plant_from_mont(zetas_inv_mont, 1)
zetas_plant = init_ntt_c_plant_from_mont(zetas_mont, 0)
zetas_inv_plant=init_ntt_c_plant_from_mont(zetas_inv_mont,1)
del zetas_plant[0]
print(len(zetas_plant), zetas_plant) # can delete the first one
print(len(zetas_inv_plant), zetas_inv_plant)



255 [3912496573, 1289999942, 1113302941, 2478231358, 4214447144, 1401833487, 3536176694, 3844278110, 4263094736, 3069830811, 4288257284, 3937099952, 3779414654, 3993575893, 1699869885, 1555045444, 616202833, 779479809, 129726913, 829804904, 1678621511, 1531001232, 353394003, 2833862031, 3706722850, 1500806174, 3719024540, 1203328945, 767178119, 3644096065, 2874122107, 3567490086, 39141741, 2333406917, 234291277, 1128400470, 847139104, 2950168918, 1082548716, 2698543442, 1928010316, 568114409, 2214304192, 3796748853, 1845812661, 1251417369, 1292795781, 2231079223, 2811495322, 469700890, 139791932, 1042288640, 105682701, 1575734650, 3639063555, 2522405608, 801846518, 1661287312, 1977217076, 2132106536, 2628647476, 674915445, 1683654021, 223667091, 3543445874, 725240540, 3232548619, 2386527851, 914798399, 50325096, 1548894599, 586566944, 4213887976, 2836098702, 435032491, 116866055, 4074096045, 24603380, 2666111713, 3499271624, 3188933536, 1188790584, 2705812622, 2719791815, 3059206624, 5

In [26]:
# test zetas success!
def test_final_zetas_inv_plant():
	for a in range(0, Q):
		am = mont_mul(zetas_inv_mont[-1], a)
		ap = plant_mul_ntt(zetas_inv_plant[-2], a)
		if((am % Q) != (ap % Q)):
			print("error!", am, ap)
			break
	print("success!!")
# test_final_zetas_inv_plant()


t = ModReverse(256, Q)
t128 = ModReverse(128, Q)
t1 = MONT*(MONT*(Q-1)*((Q-1)/256) % Q) % Q
print("MONT^2*ModReverse(256,Q)", MONT*MONT*t % Q, MONT*MONT*t128 % Q, t1)

# const24=MONT^2*256^-1
# const25=MONT^2*128^-1
const24 = (1 << 24) % Q
const25 = (1 << 25) % Q
print("1<<24", const24)
print("1<<25", const25)


MONT^2*ModReverse(256,Q) 1912 3824 1912.0
1<<24 1912
1<<25 3824


In [45]:
# NTT based on Plantard word size arithmetic
def CT_butterfly_plant(a, b, zeta):
	t = plant_mul_ntt(b, zeta)
	b = np.int16(a-t) % Q
	a = np.int16(a+t) % Q
	return a, b

def ntt_plant(a):
	for j in range(0, 384):
		t = plant_mul_ntt(a[j+384], zetas_plant[0])
		a[j+384]=np.int16(a[j]+a[j+384]-t)%Q
		a[j] = np.int16(a[j]+t)%Q
	
	k=1
	len=192
	while len>=3:
		start=0
		while start<768:
			zeta = zetas_plant[k]
			k=k+1
			j=start
			while j<start+len:
				t = plant_mul_ntt(a[j+len], zeta)
				a[j+len] = np.int16(a[j]-t)%Q
				a[j] = np.int16(a[j]+t) % Q
				j=j+1
			start=j+len
		len=len>>1
	return

def invntt_plant(a):
	k = 0
	len = 3
	while len <=192:
		start = 0
		while start < 768:
			zeta = zetas_inv_plant[k]
			k = k+1
			j = start
			while j < start+len:
				t=a[j]
				a[j]=np.int16(t+a[j+len])%Q
				a[j+len] = np.int16(t-a[j+len])
				a[j+len] = plant_mul_ntt(a[j+len], zeta)
				j = j+1
			start = j+len
		len = len << 1

	for j in range(0, 384):
		t = np.int16(a[j]-a[j+384])
		a[j] = np.int16(a[j]+a[j+384])
		t = plant_mul_ntt(t, zetas_inv_plant[254])
		a[j] = np.int16(a[j]-t)
		a[j] = plant_mul_ntt(zetas_inv_plant[255], a[j])
		a[j+384] = plant_mul_ntt(zetas_inv_plant[256], t)
	return


def besemul_plant(a, b, zeta):
	c=[0,0,0]
	c[0] = plant_mul(a[2], b[1])
	c[0] = np.int16(c[0]+ plant_mul(a[1], b[2]))
	c[0] = plant_mul_ntt(c[0], zeta)
	c[0] = np.int16(c[0]+plant_mul(a[0], b[0]))%Q

	c[1] = plant_mul(a[2], b[2])
	c[1] = plant_mul_ntt(c[1], zeta)
	c[1] = np.int16(c[1]+plant_mul(a[0], b[1]))
	c[1] = np.int16(c[1]+plant_mul(a[1], b[0]))%Q

	c[2] = plant_mul(a[2], b[0])
	c[2] = np.int16(c[2]+plant_mul(a[1], b[1]))
	c[2] = np.int16(c[2]+plant_mul(a[0], b[2]))%Q
	return c
def poly_basemul_plant(a, b):
	c=[]
	for i in range(0, 768//6):
		c += besemul_plant(a[6*i:6*i+3], b[6*i:6*i+3], zetas_plant[127+i])
		c += besemul_plant(a[6*i+3:6*i+6], b[6*i+3:6*i+6], -zetas_plant[127+i])
	return c


def besemul_mont(a, b, zeta):
	c = [0, 0, 0]
	c[0] = mont_mul(a[2], b[1])
	c[0] = np.int16(c[0] + mont_mul(a[1], b[2]))
	c[0] = mont_mul(c[0], zeta)
	c[0] = np.int16(c[0]+mont_mul(a[0], b[0])) % Q

	c[1] = mont_mul(a[2], b[2])
	c[1] = mont_mul(c[1], zeta)
	c[1] = np.int16(c[1]+mont_mul(a[0], b[1]))
	c[1] = np.int16(c[1]+mont_mul(a[1], b[0])) % Q

	c[2] = mont_mul(a[2], b[0])
	c[2] = np.int16(c[2]+mont_mul(a[1], b[1]))
	c[2] = np.int16(c[2]+mont_mul(a[0], b[2])) % Q
	return c

def poly_basemul_mont(a, b):
	c = []
	for i in range(0, 768//6):
		c += besemul_mont(a[6*i:6*i+3], b[6*i:6*i+3], zetas_mont[128+i])
		c += besemul_mont(a[6*i+3:6*i+6], b[6*i+3:6*i+6], -zetas_mont[128+i])
	return c


In [46]:
def is_equal(a,b):
	if len(a)!=len(b):
		return False
	for i in range(0,len(a)):
		if (a[i]%Q)!=(b[i]%Q):
			print("idx={0}, a[i]={1}, b[i]={2}.".format(i,a[i],b[i]))
			return False
	return True

In [47]:
def gen_streamlined_CT_table(twiddle, N, compressed_layer, merged_layer):
    new_twiddle = []
    level_ptr = [0]
    for i in range(1, N):
        level_ptr.append((1 << i)-1)
    # print("level_ptr", level_ptr)
    start_level = 0
    for i in range(0, compressed_layer):
        for j in range(0, (1 << start_level)):
            for k in range(0, merged_layer[i]):
                for h in range(0, (1 << k)):
                    new_twiddle.append(
                        twiddle[level_ptr[start_level+k]+j*(1 << k)+h])
        start_level = start_level+merged_layer[i]
    # print("streamlined CT twiddle")
    # print(new_twiddle, len(new_twiddle))
    return new_twiddle

def gen_streamlined_INV_GS_table(twiddle, N, compressed_layer, merged_layer):
    new_twiddle = []
    level_ptr = [0]
    t = 0
    for i in range(1, N):
        t = t+(1 << (N-i))
        level_ptr.append(t)
    print("level_ptr", level_ptr)
    start_level = 0
    for i in range(0, compressed_layer):  # compressed layer
        # the first layer of each compressed layer
        for j in range(0, (1 << (N-start_level-1))//(1 << (merged_layer[i]-1))):
            # print("j=",j)
            # the merged layer in each compressed layer
            for k in range(0, merged_layer[i]):
                k1 = 1 << (merged_layer[i]-1-k)
                for h in range(0, k1):  # each twiddle in each merged layer
                    print("index:", level_ptr[start_level+k]+j*k1+h)
                    new_twiddle.append(
                        twiddle[level_ptr[start_level+k]+j*k1+h])
        start_level = start_level+merged_layer[i]
    # print("streamlined invert GS twiddle")
    new_twiddle.append(twiddle[-2])
    new_twiddle.append(twiddle[-1])
    # print(new_twiddle, len(new_twiddle))
    return new_twiddle
# new_zetas_asm=gen_streamlined_CT_table(zetas_plant, 8, 3, [3,3,2])
# print("3-layer merging:", len(new_zetas_asm), new_zetas_asm)
# new_zetas_asm = gen_streamlined_CT_table(zetas_plant, 8, 2, [4,4])
# print("4-layer merging:", len(new_zetas_asm), new_zetas_asm)

new_inv_zetas_asm=gen_streamlined_INV_GS_table(zetas_inv_plant, 8, 3, [2,3,3])
print("3-layer merging:", len(new_inv_zetas_asm), new_inv_zetas_asm)
new_inv_zetas_asm=gen_streamlined_INV_GS_table(zetas_inv_plant, 8, 2, [4,4])
print("4-layer merging:", len(new_inv_zetas_asm), new_inv_zetas_asm)


level_ptr [0, 128, 192, 224, 240, 248, 252, 254]
index: 0
index: 1
index: 128
index: 2
index: 3
index: 129
index: 4
index: 5
index: 130
index: 6
index: 7
index: 131
index: 8
index: 9
index: 132
index: 10
index: 11
index: 133
index: 12
index: 13
index: 134
index: 14
index: 15
index: 135
index: 16
index: 17
index: 136
index: 18
index: 19
index: 137
index: 20
index: 21
index: 138
index: 22
index: 23
index: 139
index: 24
index: 25
index: 140
index: 26
index: 27
index: 141
index: 28
index: 29
index: 142
index: 30
index: 31
index: 143
index: 32
index: 33
index: 144
index: 34
index: 35
index: 145
index: 36
index: 37
index: 146
index: 38
index: 39
index: 147
index: 40
index: 41
index: 148
index: 42
index: 43
index: 149
index: 44
index: 45
index: 150
index: 46
index: 47
index: 151
index: 48
index: 49
index: 152
index: 50
index: 51
index: 153
index: 52
index: 53
index: 154
index: 54
index: 55
index: 155
index: 56
index: 57
index: 156
index: 58
index: 59
index: 157
index: 60
index: 61
index: 158


In [48]:
# Test NTT with Montgomery and Plantard arithmetic
def test_ntt_intt():
	for loop in range(0,100):
		am = []
		ap = []
		for i in range(0, 768):
			tmp=random.randint(0,Q)
			am.append(tmp)
			ap.append(tmp)
		ntt_mont(am)
		ntt_plant(ap)
		# in normal domain
		if(is_equal(ap,am)==False):
			print("NTT false!")
			return
		for i in range(0, len(am)):
			am[i] = mont_mul(am[i], 1)
			ap[i] = plant_mul(ap[i], 1)
		# after base mul: in montgomery domain
		invntt_mont(am)
		invntt_plant(ap)
		if(is_equal(ap, am) == False):
			print("Invert NTT false!")
			return
	print("NTT and INTT sucess!")

def test_basemul_plant():
	bp = []
	ap = []
	cp = []
	cm = []
	cp_c = [730, 104, 1062, 2224, 1539, 5968, 279, 4386, 1136, 3375, 2499, 2046, 5663, 1097, 3246, 1469, 5317, 5089, 3564, 3606, 7410, 904, 5613, 6553, 836, 3482, 2992, 3662, 5106, 366, 207, 4613, 2148, 4231, 2284, 2665, 1237, 5197, 5395, 2755, 3035, 2252, 2519, 6492, 5914, 197, 2503, 4147, 198, 4416, 3684, 3670, 6743, 5954, 6654, 4541, 5906, 2654, 3439, 2994, 5647, 5122, 150, 808, 1646, 4601, 5688, 902, 5310, 1900, 3752, 3391, 2596, 1577, 5542, 615, 2240, 1866, 6931, 947, 5389, 1186, 7256, 4524, 513, 5496, 7532, 3832, 2436, 7083, 1593, 5071, 6858, 3255, 617, 1400, 4300, 6444, 6846, 844, 4456, 3395, 5095, 6358, 5637, 935, 203, 3112, 4606, 3275, 450, 2874, 6488, 1492, 3836, 50, 4754, 6688, 1101, 2275, 2016, 6906, 5346, 2016, 3737, 1451, 7159, 3444, 236, 4220, 1942, 7635, 2420, 6497, 669, 5497, 4039, 1657, 4789, 4840, 5217, 4055, 5970, 7177, 2096, 1426, 677, 2139, 7233, 1367, 4018, 3713, 982, 2564, 1246, 1811, 3470, 5056, 2748, 2503, 3899, 4510, 537, 6999, 5250, 4331, 7598, 4717, 2730, 5355, 4881, 2033, 4245, 7290, 6584, 5631, 4457, 4949, 3520, 1275, 3094, 6958, 3270, 1216, 1499, 1524, 5724, 3932, 4151, 3902, 1025, 1276, 2408, 3284, 6878, 1742, 4785, 6768, 7115, 1955, 3844, 3682, 6354, 6858, 5872, 1216, 2608, 1365, 1280, 3236, 2650, 3757, 1239, 5932, 2112, 700, 4422, 1381, 7136, 974, 2446, 6830, 2911, 7339, 6283, 6772, 3834, 6428, 6687, 4968, 3585, 6273, 5298, 6353, 125, 2721, 2561, 512, 7549, 3500, 3763, 5249, 673, 7317, 4181, 3261, 7515, 2781, 1456, 5739, 5231, 6078, 4523, 3349, 6833, 255, 830, 6742, 6706, 1663, 1029, 7627, 952, 3316, 4296, 3501, 1305, 7219, 5547, 2298, 3875, 4944, 3119, 3027, 5193, 7424, 1083, 2725, 6809, 2101, 4277, 7552, 2871, 5463, 2463, 5908, 7283, 1730, 4102, 494, 2688, 2929, 1818, 5852, 7278, 3396, 3485, 2951, 1852, 3940, 1740, 896, 5240, 5984, 3477, 4591, 761, 4351, 4553, 7200, 6940, 5896, 1317, 5357, 4561, 6922, 2608, 5163, 4194, 3824, 5119, 5321, 7482, 5525, 940, 2587, 5413, 1204, 2048, 7441, 5441, 7678, 4471, 5768, 746, 1113, 3544, 7652, 4560, 776, 7097, 4162, 5432, 6155, 2003, 1627, 4022, 4223, 6797, 7289, 524, 5415, 4574, 6989, 4705, 7272, 6017, 2348, 98, 4509, 3921, 907, 5685, 4481, 2860, 1978, 3640, 3668, 2986, 7149, 2875, 6674, 492, 2837, 5380, 1344, 5675, 4972, 6069, 6809, 3008, 3018, 4951,
         7030, 2573, 6960, 6492, 2518, 3065, 1623, 4184, 6405, 2350, 658, 3043, 2245, 3010, 7635, 143, 1482, 1385, 4255, 5771, 2122, 1402, 6237, 7013, 660, 4827, 766, 1224, 743, 1485, 1180, 2724, 951, 5274, 1828, 4488, 3777, 2549, 3589, 7462, 2892, 7464, 7622, 2180, 3455, 6200, 3657, 3096, 6384, 924, 7505, 3679, 5762, 3743, 7324, 4774, 3726, 2366, 7515, 5718, 2883, 6288, 7653, 4748, 7675, 2921, 7204, 4744, 1766, 3828, 2337, 329, 7669, 6757, 2965, 1647, 6599, 2560, 3818, 5072, 6291, 1930, 5522, 131, 5099, 4715, 2747, 930, 3272, 6724, 4711, 4780, 4171, 7117, 5849, 107, 6274, 4014, 6849, 4041, 4328, 4837, 3218, 7196, 7171, 1490, 4799, 2236, 1471, 5905, 978, 524, 4004, 447, 1528, 2101, 968, 1802, 4921, 6369, 6409, 4209, 6964, 3368, 5199, 6415, 769, 208, 1132, 2569, 147, 3371, 7358, 2296, 1887, 4344, 1720, 6248, 727, 813, 3290, 780, 3460, 4023, 5074, 5606, 1779, 3942, 1766, 4149, 2696, 2632, 14, 3197, 376, 6291, 3305, 4767, 3616, 4654, 3388, 5937, 4902, 5897, 1357, 5256, 2631, 192, 5689, 1277, 2219, 758, 6313, 365, 2800, 3015, 428, 7449, 4995, 6939, 3150, 5638, 2193, 5696, 1731, 4558, 2987, 2865, 503, 3909, 2609, 4059, 3453, 4610, 4154, 1815, 2560, 7667, 6692, 391, 2951, 2000, 6456, 850, 6288, 1709, 7589, 4438, 4198, 3920, 5600, 6650, 1341, 85, 4471, 3823, 3437, 912, 5697, 527, 6703, 5791, 1863, 1576, 2921, 6514, 6569, 3470, 7337, 5897, 7119, 6179, 3492, 7415, 7496, 4717, 1235, 6590, 6157, 245, 818, 186, 6083, 5393, 2558, 4776, 7103, 2142, 5060, 840, 3421, 1986, 2409, 3822, 6712, 7373, 4461, 5775, 6291, 4214, 204, 2061, 5556, 6302, 4343, 6940, 6506, 4937, 6406, 161, 3726, 4962, 1384, 1301, 1279, 4354, 4833, 2186, 3125, 697, 3711, 1285, 4800, 2540, 1258, 5790, 3306, 5617, 1334, 3670, 4413, 6444, 6680, 6345, 4800, 221, 3210, 5795, 6451, 5622, 65, 5050, 335, 4429, 7409, 5450, 7131, 5763, 282, 4995, 7314, 6921, 5511, 3769, 5225, 7085, 6683, 3926, 2535, 433, 884, 3474, 5582, 5449, 5747, 4481, 3173, 4018, 4598, 2308, 3233, 7410, 6430, 2311, 4050, 1210, 7106, 3069, 1332, 1950, 3016, 3017, 1592, 3130, 4562, 7537, 3992, 1012, 4081, 5410, 574, 3184, 1207, 6394, 7498, 3707, 3809, 7008, 2941, 813, 1764, 2301, 4549, 895, 2850, 4608, 6360, 3400, 3627, 2871, 6976, 4777, 3765, 2516, 5515, 2635, 1725, 6442, 6667, 5719, 2976, 1588, 1025, 5105, 2121]
	cp_asm = [730, 104, 1062, 2224, 1539, 5968, 279, 4386, 1136, 3375, 2499, 2046, 5663, 1097, 3246, 1469, 5317, 5089, 3564, 3606, 7410, 904, 5613, 6553, 836, 3482, 2992, 3662, 5106, 366, 207, 4613, 2148, 4231, 2284, 2665, 1237, 5197, 5395, 2755, 3035, 2252, 2519, 6492, 5914, 197, 2503, 4147, 198, 4416, 3684, 3670, 6743, 5954, 6654, 4541, 5906, 2654, 3439, 2994, 5647, 5122, 150, 808, 1646, 4601, 5688, 902, 5310, 1900, 3752, 3391, 2596, 1577, 5542, 615, 2240, 1866, 6931, 947, 5389, 1186, 7256, 4524, 513, 5496, 7532, 3832, 2436, 7083, 1593, 5071, 6858, 3255, 617, 1400, 4300, 6444, 6846, 844, 4456, 3395, 5095, 6358, 5637, 935, 203, 3112, 4606, 3275, 450, 2874, 6488, 1492, 3836, 50, 4754, 6688, 1101, 2275, 2016, 6906, 5346, 2016, 3737, 1451, 7159, 3444, 236, 4220, 1942, 7635, 2420, 6497, 669, 5497, 4039, 1657, 4789, 4840, 5217, 4055, 5970, 7177, 2096, 1426, 677, 2139, 7233, 1367, 4018, 3713, 982, 2564, 1246, 1811, 3470, 5056, 2748, 2503, 3899, 4510, 537, 6999, 5250, 4331, 7598, 4717, 2730, 5355, 4881, 2033, 4245, 7290, 6584, 5631, 4457, 4949, 3520, 1275, 3094, 6958, 3270, 1216, 1499, 1524, 5724, 3932, 4151, 3902, 1025, 1276, 2408, 3284, 6878, 1742, 4785, 6768, 7115, 1955, 3844, 3682, 6354, 6858, 5872, 1216, 2608, 1365, 1280, 3236, 2650, 3757, 1239, 5932, 2112, 700, 4422, 1381, 7136, 974, 2446, 6830, 2911, 7339, 6283, 6772, 3834, 6428, 6687, 4968, 3585, 6273, 5298, 6353, 125, 2721, 2561, 512, 7549, 3500, 3763, 5249, 673, 7317, 4181, 3261, 7515, 2781, 1456, 5739, 5231, 6078, 4523, 3349, 6833, 255, 830, 6742, 6706, 1663, 1029, 7627, 952, 3316, 4296, 3501, 1305, 7219, 5547, 2298, 3875, 4944, 3119, 3027, 5193, 7424, 1083, 2725, 6809, 2101, 4277, 7552, 2871, 5463, 2463, 5908, 7283, 1730, 4102, 494, 2688, 2929, 1818, 5852, 7278, 3396, 3485, 2951, 1852, 3940, 1740, 896, 5240, 5984, 3477, 4591, 761, 4351, 4553, 7200, 6940, 5896, 1317, 5357, 4561, 6922, 2608, 5163, 4194, 3824, 5119, 5321, 7482, 5525, 940, 2587, 5413, 1204, 2048, 7441, 5441, 7678, 4471, 5768, 746, 1113, 3544, 7652, 4560, 776, 7097, 4162, 5432, 6155, 2003, 1627, 4022, 4223, 6797, 7289, 524, 5415, 4574, 6989, 4705, 7272, 6017, 2348, 98, 4509, 3921, 907, 5685, 4481, 2860, 1978, 3640, 3668, 2986, 7149, 2875, 6674, 492, 2837, 5380, 1344, 5675, 4972, 6069, 6809, 3008, 3018, 4951,
           7030, 2573, 6960, 6492, 2518, 3065, 1623, 4184, 6405, 2350, 658, 3043, 2245, 3010, 7635, 143, 1482, 1385, 4255, 5771, 2122, 1402, 6237, 7013, 660, 4827, 766, 1224, 743, 1485, 1180, 2724, 951, 5274, 1828, 4488, 3777, 2549, 3589, 7462, 2892, 7464, 7622, 2180, 3455, 6200, 3657, 3096, 6384, 924, 7505, 3679, 5762, 3743, 7324, 4774, 3726, 2366, 7515, 5718, 2883, 6288, 7653, 4748, 7675, 2921, 7204, 4744, 1766, 3828, 2337, 329, 7669, 6757, 2965, 1647, 6599, 2560, 3818, 5072, 6291, 1930, 5522, 131, 5099, 4715, 2747, 930, 3272, 6724, 4711, 4780, 4171, 7117, 5849, 107, 6274, 4014, 6849, 4041, 4328, 4837, 3218, 7196, 7171, 1490, 4799, 2236, 1471, 5905, 978, 524, 4004, 447, 1528, 2101, 968, 1802, 4921, 6369, 6409, 4209, 6964, 3368, 5199, 6415, 769, 208, 1132, 2569, 147, 3371, 7358, 2296, 1887, 4344, 1720, 6248, 727, 813, 3290, 780, 3460, 4023, 5074, 5606, 1779, 3942, 1766, 4149, 2696, 2632, 14, 3197, 376, 6291, 3305, 4767, 3616, 4654, 3388, 5937, 4902, 5897, 1357, 5256, 2631, 192, 5689, 1277, 2219, 758, 6313, 365, 2800, 3015, 428, 7449, 4995, 6939, 3150, 5638, 2193, 5696, 1731, 4558, 2987, 2865, 503, 3909, 2609, 4059, 3453, 4610, 4154, 1815, 2560, 7667, 6692, 391, 2951, 2000, 6456, 850, 6288, 1709, 7589, 4438, 4198, 3920, 5600, 6650, 1341, 85, 4471, 3823, 3437, 912, 5697, 527, 6703, 5791, 1863, 1576, 2921, 6514, 6569, 3470, 7337, 5897, 7119, 6179, 3492, 7415, 7496, 4717, 1235, 6590, 6157, 245, 818, 186, 6083, 5393, 2558, 4776, 7103, 2142, 5060, 840, 3421, 1986, 2409, 3822, 6712, 7373, 4461, 5775, 6291, 4214, 204, 2061, 5556, 6302, 4343, 6940, 6506, 4937, 6406, 161, 3726, 4962, 1384, 1301, 1279, 4354, 4833, 2186, 3125, 697, 3711, 1285, 4800, 2540, 1258, 5790, 3306, 5617, 1334, 3670, 4413, 6444, 6680, 6345, 4800, 221, 3210, 5795, 6451, 5622, 65, 5050, 335, 4429, 7409, 5450, 7131, 5763, 282, 4995, 7314, 6921, 5511, 3769, 5225, 7085, 6683, 3926, 2535, 433, 884, 3474, 5582, 5449, 5747, 4481, 3173, 4018, 4598, 2308, 3233, 7410, 6430, 2311, 4050, 1210, 7106, 3069, 1332, 1950, 3016, 3017, 1592, 3130, 4562, 7537, 3992, 1012, 4081, 5410, 574, 3184, 1207, 6394, 7498, 3707, 3809, 7008, 2941, 813, 1764, 2301, 4549, 895, 2850, 4608, 6360, 3400, 3627, 2871, 6976, 4777, 3765, 2516, 5515, 2635, 1725, 6442, 6667, 5719, 2976, 1588, 1025, 5105, 2121]
	print(is_equal(cp_c,cp_asm))
	for i in range(0, 768):
		bp.append(i)
		ap.append(i)
	ntt_plant(bp)
	ntt_plant(ap)

	cp=poly_basemul_plant(ap,bp)
	# cm=poly_basemul_mont(ap,bp)
	for i in range(0,768):
		cp[i]=(cp[i]*PLANT)%Q
		# cm[i] = (cm[i]*MONT) % Q

	invntt_plant(cp)
	print("INTT:", cp)

def test_ntt_assembly():
	am = []
	ap = []
	ac =[ 7255, 7376, 7497, 7618, 58, 179, 300, 421, 542, 663, 784, 905, 1026, 1147, 1268, 1389, 1510, 1631, 1752, 1873, 1994, 2115, 2236, 2357, 2478, 2599, 2720, 2841, 2962, 3083, 3204, 3325, 3446, 3567, 3688, 3809, 3930, 4051, 4172, 4293, 4414, 4535, 4656, 4777, 4898, 5019, 5140, 5261, 3536, 1764, 7673, 5901, 4129, 2357, 585, 6494, 4722, 2950, 1178, 7087, 5315, 3543, 1771, 7680, 5908, 4136, 2364, 592, 6501, 4729, 2957, 1185, 7094, 5322, 3550, 1778, 6, 5915, 4143, 2371, 599, 6508, 4736, 2964, 1192, 7101, 5329, 3557, 1785, 13, 5922, 4150, 2378, 606, 6515, 4743, 243, 2059, 3875, 5691, 7507, 1642, 3458, 5274, 7090, 1225, 3041, 4857, 6673, 808, 2624, 4440, 6256, 391, 2207, 4023, 5839, 7655, 1790, 3606, 5422, 7238, 1373, 3189, 5005, 6821, 956, 2772, 4588, 6404, 539, 2355, 4171, 5987, 122, 1938, 3754, 5570, 7386, 1521, 3337, 5153, 6969, 1104, 3113, 4524, 5935, 7346, 1076, 2487, 3898, 5309, 6720, 450, 1861, 3272, 4683, 6094, 7505, 1235, 2646, 4057, 5468, 6879, 609, 2020, 3431, 4842, 6253, 7664, 1394, 2805, 4216, 5627, 7038, 768, 2179, 3590, 5001, 6412, 142, 1553, 2964, 4375, 5786, 7197, 927, 2338, 3749, 5160, 6571, 301, 740, 7572, 6723, 5874, 5025, 4176, 3327, 2478, 1629, 780, 7612, 6763, 5914, 5065, 4216, 3367, 2518, 1669, 820, 7652, 6803, 5954, 5105, 4256, 3407, 2558, 1709, 860, 11, 6843, 5994, 5145, 4296, 3447, 2598, 1749, 900, 51, 6883, 6034, 5185, 4336, 3487, 2638, 1789, 940, 91, 6923, 791, 4629, 786, 4624, 781, 4619, 776, 4614, 771, 4609, 766, 4604, 761, 4599, 756, 4594, 751, 4589, 746, 4584, 741, 4579, 736, 4574, 731, 4569, 726, 4564, 721, 4559, 716, 4554, 711, 4549, 706, 4544, 701, 4539, 696, 4534, 691, 4529, 686, 4524, 681, 4519, 676, 4514, 4024, 5036, 6048, 7060, 391, 1403, 2415, 3427, 4439, 5451, 6463, 7475, 806, 1818, 2830, 3842, 4854, 5866, 6878, 209, 1221, 2233, 3245, 4257, 5269, 6281, 7293, 624, 1636, 2648, 3660, 4672, 5684, 6696, 27, 1039, 2051, 3063, 4075, 5087, 6099, 7111, 442, 1454, 2466, 3478, 4490, 5502, 4181, 2985, 1789, 593, 7078, 5882, 4686, 3490, 2294, 1098, 7583, 6387, 5191, 3995, 2799, 1603, 407, 6892, 5696, 4500, 3304, 2108, 912, 7397, 6201, 5005, 3809, 2613, 1417, 221, 6706, 5510, 4314, 3118, 1922, 726, 7211, 6015, 4819, 3623, 2427, 1231, 35, 6520, 5324, 4128, 2932, 1736, 7343, 6893, 6443, 5993, 5543, 5093, 4643, 4193, 3743, 3293, 2843, 2393, 1943, 1493, 1043, 593, 143, 7374, 6924, 6474, 6024, 5574, 5124, 4674, 4224, 3774, 3324, 2874, 2424, 1974, 1524, 1074, 624, 174, 7405, 6955, 6505, 6055, 5605, 5155, 4705, 4255, 3805, 3355, 2905, 2455, 2005, 1555, 3645, 310, 4656, 1321, 5667, 2332, 6678, 3343, 8, 4354, 1019, 5365, 2030, 6376, 3041, 7387, 4052, 717, 5063, 1728, 6074, 2739, 7085, 3750, 415, 4761, 1426, 5772, 2437, 6783, 3448, 113, 4459, 1124, 5470, 2135, 6481, 3146, 7492, 4157, 822, 5168, 1833, 6179, 2844, 7190, 3855, 520, 4688, 6406, 443, 2161, 3879, 5597, 7315, 1352, 3070, 4788, 6506, 543, 2261, 3979, 5697, 7415, 1452, 3170, 4888, 6606, 643, 2361, 4079, 5797, 7515, 1552, 3270, 4988, 6706, 743, 2461, 4179, 5897, 7615, 1652, 3370, 5088, 6806, 843, 2561, 4279, 5997, 34, 1752, 3470, 5188, 6906, 943, 692, 7160, 5947, 4734, 3521, 2308, 1095, 7563, 6350, 5137, 3924, 2711, 1498, 285, 6753, 5540, 4327, 3114, 1901, 688, 7156, 5943, 4730, 3517, 2304, 1091, 7559, 6346, 5133, 3920, 2707, 1494, 281, 6749, 5536, 4323, 3110, 1897, 684, 7152, 5939, 4726, 3513, 2300, 1087, 7555, 6342, 5129, 4968, 5714, 6460, 7206, 271, 1017, 1763, 2509, 3255, 4001, 4747, 5493, 6239, 6985, 50, 796, 1542, 2288, 3034, 3780, 4526, 5272, 6018, 6764, 7510, 575, 1321, 2067, 2813, 3559, 4305, 5051, 5797, 6543, 7289, 354, 1100, 1846, 2592, 3338, 4084, 4830, 5576, 6322, 7068, 133, 879, 1625, 4734, 5984, 7234, 803, 2053, 3303, 4553, 5803, 7053, 622, 1872, 3122, 4372, 5622, 6872, 441, 1691, 2941, 4191, 5441, 6691, 260, 1510, 2760, 4010, 5260, 6510, 79, 1329, 2579, 3829, 5079, 6329, 7579, 1148, 2398, 3648, 4898, 6148, 7398, 967, 2217, 3467, 4717, 5967, 7217, 786, 2036, 946, 5924, 3221, 518, 5496, 2793, 90, 5068, 2365, 7343, 4640, 1937, 6915, 4212, 1509, 6487, 3784, 1081, 6059, 3356, 653, 5631, 2928, 225, 5203, 2500, 7478, 4775, 2072, 7050, 4347, 1644, 6622, 3919, 1216, 6194, 3491, 788, 5766, 3063, 360, 5338, 2635, 7613, 4910, 2207, 7185, 4482, 2193, 3894, 5595, 7296, 1316, 3017, 4718, 6419, 439, 2140, 3841, 5542, 7243, 1263, 2964, 4665, 6366, 386, 2087, 3788, 5489, 7190, 1210, 2911, 4612, 6313, 333, 2034, 3735, 5436, 7137, 1157, 2858, 4559, 6260, 280, 1981, 3682, 5383, 7084, 1104, 2805, 4506, 6207, 227, 1928, 3629, 5330]
	j=-7681//2
	for i in range(0, 768):
		am.append(j%7681)
		ap.append(j%7681)
		j+=1
	print(ap)
	ntt_mont(am)
	ntt_plant(ap)
	# in normal domain
	if(is_equal(ap,ac) == False):
		print("NTT false!",ap,"\n",ac)
		# return
	else:
		print("NTT success!")
	for i in range(0, len(am)):
		am[i] = mont_mul(am[i], 1)
		ap[i] = plant_mul(ap[i], 1)
	# after base mul: in montgomery domain
	invntt_mont(am)
	invntt_plant(ap)

	if(is_equal(ap, am) == False):
		print("Invert NTT false!", ap,"\n",am)
		return
	else:
		print("Invert NTT success!", ap, "\n", am)

# test_ntt_intt()
# test_basemul_plant()
test_ntt_assembly()


[3840, 3841, 3842, 3843, 3844, 3845, 3846, 3847, 3848, 3849, 3850, 3851, 3852, 3853, 3854, 3855, 3856, 3857, 3858, 3859, 3860, 3861, 3862, 3863, 3864, 3865, 3866, 3867, 3868, 3869, 3870, 3871, 3872, 3873, 3874, 3875, 3876, 3877, 3878, 3879, 3880, 3881, 3882, 3883, 3884, 3885, 3886, 3887, 3888, 3889, 3890, 3891, 3892, 3893, 3894, 3895, 3896, 3897, 3898, 3899, 3900, 3901, 3902, 3903, 3904, 3905, 3906, 3907, 3908, 3909, 3910, 3911, 3912, 3913, 3914, 3915, 3916, 3917, 3918, 3919, 3920, 3921, 3922, 3923, 3924, 3925, 3926, 3927, 3928, 3929, 3930, 3931, 3932, 3933, 3934, 3935, 3936, 3937, 3938, 3939, 3940, 3941, 3942, 3943, 3944, 3945, 3946, 3947, 3948, 3949, 3950, 3951, 3952, 3953, 3954, 3955, 3956, 3957, 3958, 3959, 3960, 3961, 3962, 3963, 3964, 3965, 3966, 3967, 3968, 3969, 3970, 3971, 3972, 3973, 3974, 3975, 3976, 3977, 3978, 3979, 3980, 3981, 3982, 3983, 3984, 3985, 3986, 3987, 3988, 3989, 3990, 3991, 3992, 3993, 3994, 3995, 3996, 3997, 3998, 3999, 4000, 4001, 4002, 4003, 4004, 4005, 400

In [49]:
# for details, please see NTTRU paper
# X^768 - X^384 + 1 = (X^384 - zeta^128)(X^384 - zeta^640), so the first twiddle factor is zeta^128
# = (X^192 - zeta^64)(X^192 + zeta^64)(X^192 - zeta^320)(X^192 + zeta^320), so the twiddle factors is zeta^64, zeta^320

polys_degree = 768
zetas_degree = []
n = 768
n_half = 768 //2

# we will factor the polynomial 8th into X^3 - zeta^?
# we will get the degree of zeta used when computing NTT of NTTRU
for i in range(0, 8):
    if i == 0:
        zetas_degree.append(128)
    elif i == 1:
        zetas_degree.append(64)
        zetas_degree.append(320)
    else:
        len_t = len(zetas_degree)
        for ii in range(len_t // 2, len_t):
            d = zetas_degree[ii]
            zetas_degree.append(d // 2)
            zetas_degree.append((d + n_half) // 2)

print(zetas_degree)
print("len(zetas_degree) is ", len(zetas_degree))

[128, 64, 320, 32, 224, 160, 352, 16, 208, 112, 304, 80, 272, 176, 368, 8, 200, 104, 296, 56, 248, 152, 344, 40, 232, 136, 328, 88, 280, 184, 376, 4, 196, 100, 292, 52, 244, 148, 340, 28, 220, 124, 316, 76, 268, 172, 364, 20, 212, 116, 308, 68, 260, 164, 356, 44, 236, 140, 332, 92, 284, 188, 380, 2, 194, 98, 290, 50, 242, 146, 338, 26, 218, 122, 314, 74, 266, 170, 362, 14, 206, 110, 302, 62, 254, 158, 350, 38, 230, 134, 326, 86, 278, 182, 374, 10, 202, 106, 298, 58, 250, 154, 346, 34, 226, 130, 322, 82, 274, 178, 370, 22, 214, 118, 310, 70, 262, 166, 358, 46, 238, 142, 334, 94, 286, 190, 382, 1, 193, 97, 289, 49, 241, 145, 337, 25, 217, 121, 313, 73, 265, 169, 361, 13, 205, 109, 301, 61, 253, 157, 349, 37, 229, 133, 325, 85, 277, 181, 373, 7, 199, 103, 295, 55, 247, 151, 343, 31, 223, 127, 319, 79, 271, 175, 367, 19, 211, 115, 307, 67, 259, 163, 355, 43, 235, 139, 331, 91, 283, 187, 379, 5, 197, 101, 293, 53, 245, 149, 341, 29, 221, 125, 317, 77, 269, 173, 365, 17, 209, 113, 305, 65, 2

In [50]:
# cross-validation: the tree list is got from NTTRU's source code
tree_nttru = [1, 128, 64, 320, 32, 224, 160, 352, 16, 208, 112, 304, 80, 272, 176, 368, 8, 200, 104, 296, 56, 248, 152, 344, 40, 232, 136, 328, 88, 280, 184, 376, 4, 196, 100, 292, 52, 244, 148, 340, 28, 220, 124, 316, 76, 268, 172, 364, 20, 212, 116, 308, 68, 260, 164, 356, 44, 236, 140, 332, 92, 284, 188, 380, 2, 194, 98, 290, 50, 242, 146, 338, 26, 218, 122, 314, 74, 266, 170, 362, 14, 206, 110, 302, 62, 254, 158, 350, 38, 230, 134, 326, 86, 278, 182, 374, 10, 202, 106, 298, 58, 250, 154, 346, 34, 226, 130, 322, 82, 274, 178, 370, 22, 214, 118, 310, 70, 262, 166, 358, 46, 238, 142, 334, 94, 286, 190, 382, 1, 193, 97, 289, 49, 241, 145, 337, 25, 217, 121, 313, 73, 265, 169, 361, 13, 205, 109, 301, 61, 253, 157, 349, 37, 229, 133, 325, 85, 277, 181, 373, 7, 199, 103, 295, 55, 247, 151, 343, 31, 223, 127, 319, 79, 271, 175, 367, 19, 211, 115, 307, 67, 259, 163, 355, 43, 235, 139, 331, 91, 283, 187, 379, 5, 197, 101, 293, 53, 245, 149, 341, 29, 221, 125, 317, 77, 269, 173, 365, 17, 209, 113, 305, 65, 257, 161, 353, 41, 233, 137, 329, 89, 281, 185, 377, 11, 203, 107, 299, 59, 251, 155, 347, 35, 227, 131, 323, 83, 275, 179, 371, 23, 215, 119, 311, 71, 263, 167, 359, 47, 239, 143, 335, 95, 287, 191, 383]

normal_degree = [i for i in range(0, 384)]
nttru_zeta_degree = []

for i in range(0, 256):
    nttru_zeta_degree.append(normal_degree[tree_nttru[i]])

print(nttru_zeta_degree)

for i in range(0, 255):
    if zetas_degree[i] != nttru_zeta_degree[i+1]:
        print("error")

[1, 128, 64, 320, 32, 224, 160, 352, 16, 208, 112, 304, 80, 272, 176, 368, 8, 200, 104, 296, 56, 248, 152, 344, 40, 232, 136, 328, 88, 280, 184, 376, 4, 196, 100, 292, 52, 244, 148, 340, 28, 220, 124, 316, 76, 268, 172, 364, 20, 212, 116, 308, 68, 260, 164, 356, 44, 236, 140, 332, 92, 284, 188, 380, 2, 194, 98, 290, 50, 242, 146, 338, 26, 218, 122, 314, 74, 266, 170, 362, 14, 206, 110, 302, 62, 254, 158, 350, 38, 230, 134, 326, 86, 278, 182, 374, 10, 202, 106, 298, 58, 250, 154, 346, 34, 226, 130, 322, 82, 274, 178, 370, 22, 214, 118, 310, 70, 262, 166, 358, 46, 238, 142, 334, 94, 286, 190, 382, 1, 193, 97, 289, 49, 241, 145, 337, 25, 217, 121, 313, 73, 265, 169, 361, 13, 205, 109, 301, 61, 253, 157, 349, 37, 229, 133, 325, 85, 277, 181, 373, 7, 199, 103, 295, 55, 247, 151, 343, 31, 223, 127, 319, 79, 271, 175, 367, 19, 211, 115, 307, 67, 259, 163, 355, 43, 235, 139, 331, 91, 283, 187, 379, 5, 197, 101, 293, 53, 245, 149, 341, 29, 221, 125, 317, 77, 269, 173, 365, 17, 209, 113, 305, 65